In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
import math

In [6]:
input = Input((300,300,3))
x = Conv2D(filters=12, kernel_size=(3,3), activation='relu')(input)
x = Flatten()(x)
output = Dense(1,  activation='linear')(x)

In [7]:
model = Model(input, output)
model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 298, 298, 12)      336       
_________________________________________________________________
flatten_2 (Flatten)          (None, 1065648)           0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1065649   
Total params: 1,065,985
Trainable params: 1,065,985
Non-trainable params: 0
_________________________________________________________________


In [19]:
# tf.keras.optimizers.Adam(
#     learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
#     name='Adam', **kwargs
# )

# Lets tweak learning rate to see rate of conversion
optimizer = Adam(learning_rate=0.002);

model.compile(loss=MeanSquaredError(), optimizer=Adam(), metrics=['mae'])

reduce_lr = ReduceLROnPlateau(monitor='accuracy', patience=6, verbose=1, factor=0.75, min_lr=0.00005)

early_stop = EarlyStopping(
    monitor="accuracy",
    min_delta=0.00005,
    patience=10,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [20]:
df = pd.read_csv('./petfinder-pawpularity-score/train.csv')

In [21]:
df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [22]:
type(df['Pawpularity'][0])

numpy.int64

In [23]:
df['Id']

0       0007de18844b0dbbb5e1f607da0606e0
1       0009c66b9439883ba2750fb825e1d7db
2       0013fd999caf9a3efe1352ca1b0d937e
3       0018df346ac9c1d8413cfcc888ca8246
4       001dc955e10590d3ca4673f034feeef2
                      ...               
9907    ffbfa0383c34dc513c95560d6e1fdb57
9908    ffcc8532d76436fc79e50eb2e5238e45
9909    ffdf2e8673a1da6fb80342fa3b119a20
9910    fff19e2ce11718548fa1c5d039a5192a
9911    fff8e47c766799c9e12f3cb3d66ad228
Name: Id, Length: 9912, dtype: object

In [24]:
df['Id'] = df['Id'] + '.jpg';
df['Id']

0       0007de18844b0dbbb5e1f607da0606e0.jpg
1       0009c66b9439883ba2750fb825e1d7db.jpg
2       0013fd999caf9a3efe1352ca1b0d937e.jpg
3       0018df346ac9c1d8413cfcc888ca8246.jpg
4       001dc955e10590d3ca4673f034feeef2.jpg
                        ...                 
9907    ffbfa0383c34dc513c95560d6e1fdb57.jpg
9908    ffcc8532d76436fc79e50eb2e5238e45.jpg
9909    ffdf2e8673a1da6fb80342fa3b119a20.jpg
9910    fff19e2ce11718548fa1c5d039a5192a.jpg
9911    fff8e47c766799c9e12f3cb3d66ad228.jpg
Name: Id, Length: 9912, dtype: object

In [25]:
df['Pawpularity']

0       63
1       42
2       28
3       15
4       72
        ..
9907    15
9908    70
9909    20
9910    20
9911    30
Name: Pawpularity, Length: 9912, dtype: int64

In [26]:
# You can see more informations here https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

# tf.keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False, samplewise_center=False,
#     featurewise_std_normalization=False, samplewise_std_normalization=False,
#     zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
#     height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
#     channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
#     horizontal_flip=False, vertical_flip=False, rescale=None,
#     preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
# )

train_dataGen = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    zoom_range = 0.1,
    rotation_range = 10,
    shear_range = 10/180 * math.pi
)

# flow_from_dataframe(
#     dataframe, directory=None, x_col='filename', y_col='class',
#     weight_col=None, target_size=(256, 256), color_mode='rgb',
#     classes=None, class_mode='categorical', batch_size=32, shuffle=True,
#     seed=None, save_to_dir=None, save_prefix='',
#     save_format='png', subset=None, interpolation='nearest',
#     validate_filenames=True, **kwargs
# )

# target_size	tuple of integers (height, width), default: (256, 256). The dimensions to which all images found will be resized.
# class_mode	one of "binary", "categorical", "input", "multi_output", "raw", sparse" or None. "sparse": 1D numpy array of integer labels

# TODO: We will need to pass the data as array of strings in order to use sparse!!!!

train_gen = train_dataGen.flow_from_dataframe(
    dataframe=df,
    directory='./petfinder-pawpularity-score/train',
    x_col='Id',
    y_col='Pawpularity',
    target_size=(300, 300), 
    color_mode='rgb',
    class_mode='raw',
    batch_size=32
)

Found 9912 validated image filenames.


In [ ]:
# model.fit(train_gen, epochs=10, callbacks=[tensorboard_callback, reduce_lr, early_stop])
history = model.fit(train_gen, epochs=10, verbose=1)

Epoch 1/10
310/310 [==============================] - 385s 1s/step - loss: 10605728.0000 - mae: 1020.6481
Epoch 2/10
161/310 [==============>...............] - ETA: 3:08 - loss: 6728.7246 - mae: 61.7590